<a href="https://colab.research.google.com/github/HatemMoushir/Shark-identification-1/blob/main/Shark%20identification_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os

path = "/content/drive/MyDrive/Shark_project"
files = os.listdir(path)
print(files)

['sharks_raw.zip']


In [ ]:
import os
from PIL import Image, UnidentifiedImageError
import imagehash
from collections import defaultdict
import matplotlib.pyplot as plt

base_path = "/content/drive/MyDrive/Shark_project"

# إعدادات عامة
min_size = (100, 100)
hash_dict = defaultdict(list)
duplicates = defaultdict(list)
bad_images = []
small_images = []

def display_images(image_paths, title):
    print(f"\n📸 {title} - عدد الصور: {len(image_paths)}\n")
    for i, path in enumerate(image_paths[:10]):  # نعرض فقط أول 10 صور
        try:
            img = Image.open(path).convert("RGB")
            plt.imshow(img)
            plt.title(f"{os.path.basename(path)}")
            plt.axis('off')
            plt.show()
        except Exception as e:
            print(f"❌ خطأ أثناء العرض: {path}", e)

# فحص شامل على كل الصور
for class_name in os.listdir(base_path):
    class_path = os.path.join(base_path, class_name)
    if not os.path.isdir(class_path):
        continue

    for img_name in os.listdir(class_path):
        img_path = os.path.join(class_path, img_name)
        try:
            img = Image.open(img_path)
            img.verify()  # فحص إذا كانت تالفة
            img = Image.open(img_path).convert("RGB")  # إعادة الفتح

            # فحص الحجم
            if img.size[0] < min_size[0] or img.size[1] < min_size[1]:
                small_images.append(img_path)

            # حساب phash واكتشاف التكرار
            hash_val = imagehash.phash(img)
            hash_dict[str(hash_val)].append(img_path)

        except (UnidentifiedImageError, OSError) as e:
            bad_images.append(img_path)

# معالجة التكرارات
for h, paths in hash_dict.items():
    if len(paths) > 1:
        duplicates[h] = paths

# -------------------------------
# عرض النتائج
# -------------------------------

# الصور التالفة
if bad_images:
    display_images(bad_images, "صور تالفة")

# الصور صغيرة الحجم
if small_images:
    display_images(small_images, "صور صغيرة الحجم (< 100x100)")

# الصور المكررة بصريًا
if duplicates:
    print(f"\n🔁 مجموعات صور مكررة بصريًا: {len(duplicates)}\n")
    for i, (h, paths) in enumerate(duplicates.items()):
        print(f"\n🧩 المجموعة {i+1} - Hash: {h} - عدد الصور: {len(paths)}")
        display_images(paths, f"صور مكررة - مجموعة {i+1}")